In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM